In [1]:
pip install pymysql

In [2]:
from query_sql import query_stmt
from db_connection import mydb_engine

In [3]:
from sqlalchemy import exc, text, bindparam
import pandas as pd
stmt = """ select * from eth_price;"""
t = text(stmt)
df = pd.read_sql(t, con = mydb_engine)
df

,id,datetime,open,high,low,close,volume,pct_chg,w_avg_price
0,1,2023-12-07 19:50:06,2265.27,2284.6,2220.0,2250.12,346229.8826,-0.669,2251.618661
1,2,2023-12-07 19:50:07,2265.27,2284.6,2220.0,2250.30,346234.7202,-0.661,2251.618640
2,3,2023-12-07 19:50:08,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625
3,4,2023-12-07 19:50:09,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625
4,5,2023-12-07 19:50:10,2265.27,2284.6,2220.0,2250.42,346238.9490,-0.656,2251.618625
...,...,...,...,...,...,...,...,...,...
29568,29569,2023-12-08 04:13:37,2259.78,2360.0,2220.0,2354.66,473454.9189,4.199,2285.303926
29569,29570,2023-12-08 04:13:38,2259.78,2360.0,2220.0,2354.66,473455.1169,4.199,2285.303955
29570,29571,2023-12-08 04:13:39,2259.78,2360.0,2220.0,2354.65,473457.0187,4.198,2285.304234
29571,29572,2023-12-08 04:13:40,2259.78,2360.0,2220.0,2354.65,473457.0434,4.198,2285.304237


In [4]:
# First we will import the necessary Library 
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

# For Evalution we will use these library
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler


# For PLotting we will use these library
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [5]:
df = query_stmt(mydb_engine)

In [6]:
# preprocess dataset

In [7]:
df2 = df.copy() #.set_index('datetime')

In [8]:
df2

,id,datetime,open,high,low,close,volume,pct_chg,w_avg_price
0,1,2023-12-07 19:50:06,2265.27,2284.6,2220.0,2250.12,346229.8826,-0.669,2251.618661
1,2,2023-12-07 19:50:07,2265.27,2284.6,2220.0,2250.30,346234.7202,-0.661,2251.618640
2,3,2023-12-07 19:50:08,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625
3,4,2023-12-07 19:50:09,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625
4,5,2023-12-07 19:50:10,2265.27,2284.6,2220.0,2250.42,346238.9490,-0.656,2251.618625
...,...,...,...,...,...,...,...,...,...
29568,29569,2023-12-08 04:13:37,2259.78,2360.0,2220.0,2354.66,473454.9189,4.199,2285.303926
29569,29570,2023-12-08 04:13:38,2259.78,2360.0,2220.0,2354.66,473455.1169,4.199,2285.303955
29570,29571,2023-12-08 04:13:39,2259.78,2360.0,2220.0,2354.65,473457.0187,4.198,2285.304234
29571,29572,2023-12-08 04:13:40,2259.78,2360.0,2220.0,2354.65,473457.0434,4.198,2285.304237


In [9]:
# extract date and time
df2['date'] = df2['datetime'].dt.normalize()
df2['hour'] = df2['datetime'].dt.hour
df2['min'] = df2['datetime'].dt.minute

# create Y column
df2['nxt_3m_close'] = df2['close'].shift(-180)
df2['return_chg3m'] = (df2['nxt_3m_close'] - df2['close']) / df2['close']

# drop NA
df2 = df2.loc[df2['return_chg3m'].isna() == False]
df2.head()

,id,datetime,open,high,low,close,volume,pct_chg,w_avg_price,date,hour,min,nxt_3m_close,return_chg3m
0,1,2023-12-07 19:50:06,2265.27,2284.6,2220.0,2250.12,346229.8826,-0.669,2251.618661,2023-12-07,19,50,2250.87,0.000333
1,2,2023-12-07 19:50:07,2265.27,2284.6,2220.0,2250.30,346234.7202,-0.661,2251.618640,2023-12-07,19,50,2250.87,0.000253
2,3,2023-12-07 19:50:08,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625,2023-12-07,19,50,2250.87,0.000200
3,4,2023-12-07 19:50:09,2265.27,2284.6,2220.0,2250.42,346238.9048,-0.656,2251.618625,2023-12-07,19,50,2250.87,0.000200
4,5,2023-12-07 19:50:10,2265.27,2284.6,2220.0,2250.42,346238.9490,-0.656,2251.618625,2023-12-07,19,50,2250.87,0.000200


In [10]:
len(df2)

29393

In [11]:
len(df2.loc[df2['return_chg3m'].isna() == False])

29393

In [12]:
len(df2.loc[df2['return_chg3m'].isna()])

0

In [13]:
print(df2.columns)

Index(['id', 'datetime', 'open', 'high', 'low', 'close', 'volume', 'pct_chg',
       'w_avg_price', 'date', 'hour', 'min', 'nxt_3m_close', 'return_chg3m'],
      dtype='object')


In [14]:
# Random Forest

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [16]:
#column_to_drop = ['id', 'datetime', 'volume', 'pct_chg', 'w_avg_price', 'date', 'hour', 'min', 'nxt_3m_close', 'return_chg3m']
#X = df2.drop(columns=column_to_drop, inplace=True)

columns_to_include = ['open', 'high', 'low', 'close']
X = df2[columns_to_include]

y = df2['nxt_3m_close']

In [17]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
X_train

,open,high,low,close
11778,2258.32,2341.9,2220.0,2340.39
27004,2260.86,2356.0,2220.0,2344.21
25176,2262.99,2356.0,2220.0,2335.65
22635,2269.66,2356.0,2220.0,2336.09
13227,2267.76,2353.2,2220.0,2343.23
...,...,...,...,...
21575,2272.75,2356.0,2220.0,2341.55
5390,2262.08,2284.6,2220.0,2247.73
860,2265.42,2284.6,2220.0,2255.63
15795,2279.24,2353.2,2220.0,2343.51


In [19]:
X_test

,open,high,low,close
17213,2273.10,2353.2,2220.0,2330.42
17977,2265.81,2353.2,2220.0,2334.09
11127,2254.02,2329.9,2220.0,2322.08
28890,2256.76,2356.0,2220.0,2345.49
12778,2266.86,2348.7,2220.0,2344.71
...,...,...,...,...
12789,2266.86,2348.7,2220.0,2344.62
4550,2260.20,2284.6,2220.0,2250.00
9538,2256.22,2323.8,2220.0,2312.36
3693,2260.78,2284.6,2220.0,2247.45


In [20]:
y_train

11778    2335.48
27004    2343.67
25176    2336.93
22635    2334.82
13227    2350.68
          ...   
21575    2337.23
5390     2250.59
860      2253.77
15795    2340.23
23654    2334.36
Name: nxt_3m_close, Length: 23514, dtype: float64

In [21]:
y_test

17213    2328.03
17977    2334.59
11127    2327.25
28890    2348.96
12778    2344.01
          ...   
12789    2346.33
4550     2250.09
9538     2313.38
3693     2247.77
25829    2343.85
Name: nxt_3m_close, Length: 5879, dtype: float64

In [22]:
# Check valid score names
from sklearn.metrics import get_scorer_names
print(get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weight

In [25]:
# Random Forest GridSearch

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5)

In [26]:
# Fit the model to the data
grid_search.fit(X_train, y_train)

c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_root_mean_squared_error')

In [27]:

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Get the best model
best_model = grid_search.best_estimator_

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


In [28]:
# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Mean Squared Error on Test Set: 0.41285646046568275


c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [30]:
print(f'Mean Squared Error on Test Set: {mse}')

Mean Squared Error on Test Set: 0.41285646046568275


In [31]:
# Calculate root mean squared error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 0.642539073104261


In [32]:
# Try getting RMSE in GridSearch

In [33]:
# Creat Root Mean Square Error in GridSearch

from sklearn.metrics import make_scorer

# Assuming you have a function for RMSE
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Create a scorer for RMSE
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

In [34]:
# Random Forest GridSearch

param_grid2 = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest Regressor
rf_regressor2 = RandomForestRegressor()

# Create the GridSearchCV object
grid_search2 = GridSearchCV(estimator=rf_regressor2, param_grid=param_grid2, scoring=rmse_scorer, cv=5)

In [35]:
# Fit the model to the data
grid_search2.fit(X_train, y_train)

c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring=make_scorer(root_mean_squared_error, greater_is_better=False))

In [36]:

# Get the best hyperparameters
best_params2 = grid_search2.best_params_
print(f"Best Hyperparameters: {best_params2}")

# Get the best model
best_model2 = grid_search2.best_estimator_

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


In [39]:
# Evaluate the best model on the test set
y_pred = best_model2.predict(X_test)
mse2 = mean_squared_error(y_test, y_pred)

c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\wicm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is

In [40]:
print(f'Mean Squared Error on Test Set: {mse2}')

Mean Squared Error on Test Set: 0.41233639576196196


In [41]:
# Calculate root mean squared error
rmse2 = np.sqrt(mse2)

print("Root Mean Squared Error:", rmse2)

Root Mean Squared Error: 0.6421342505753466
